In [ ]:
import os
import torch
from os import makedirs
from gaussian_renderer import render, GaussianModel
import torchvision
import sys
from scene import Scene
from utils.general_utils import safe_state
from tqdm import tqdm
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, get_combined_args
from PIL import Image
from scene.cameras import Camera
import numpy as np
from tglcourse.generation_utils import calc_vgg_features, ContentLossToTarget, CLIPLossToTargets

In [ ]:
sys.argv=["render.py", "--model_path", "../output/gang", "--source_path", "../data/gang", "--iteration", "30000", "--skip_train",
          "--skip_test", "--quiet"]
parser = ArgumentParser(description="Testing Script Parameters")
model = ModelParams(parser, sentinel=True)
pipeline = PipelineParams(parser)
parser.add_argument("--iteration", default=-1, type=int)
parser.add_argument("--skip_train", action="store_true")
parser.add_argument("--skip_test", action="store_true")
parser.add_argument("--quiet", action="store_true")
args = get_combined_args(parser)
safe_state(args.quiet)

dataset, iteration, pipeline = model.extract(args), args.iteration, pipeline.extract(args)

with torch.no_grad():
    gaussians = GaussianModel(dataset.sh_degree)
    scene = Scene(dataset, gaussians, load_iteration=iteration, shuffle=False)
    bg_color = [1,1,1] if dataset.white_background else [0, 0, 0]
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")
    views = scene.getTrainCameras()
    view = views[0]
    rendering = render(view, gaussians, pipeline, background)["render"]
    # gt = view.original_image[0:3, :, :]
#     torchvision.utils.save_image(rendering, "pic.png")
# im = Image.open("pic.png")
# im.resize((640, 480))

In [ ]:
def preview(view):
    with torch.no_grad():
        rendering = render(view, gaussians, pipeline, background)["render"]
    return Image.fromarray((rendering.cpu().numpy() * 255).astype(np.uint8).transpose(1, 2, 0)).resize((640, 480))
preview(views[8])

In [ ]:
if not os.path.exists('output_frames'):
    os.makedirs('output_frames')
from numpy import sin, cos
from IPython.display import Image as IPImage
def radians(degrees):
    return degrees * np.pi / 180

def preview_arc(view):
    start_cam = view
    initial_R = np.array(start_cam.R)
    initial_T = np.array(start_cam.T)
    images = []
    for idx, i in enumerate(list(range(12)) + list(range(10, -1, -1))):
        theta = radians(5 * i - 30)
        
        R_x = np.array([[1, 0, 0],
                        [0, cos(theta), -sin(theta)],
                        [0, sin(theta), cos(theta)]])
        R_y = np.array([[cos(theta), 0, sin(theta)],
                        [0, 1, 0],
                        [-sin(theta), 0, cos(theta)]])
        R_z = np.array([[cos(theta), -sin(theta), 0],
                        [sin(theta), cos(theta), 0],
                        [0, 0, 1]])
        
        rot = R_x * 0.2 + R_y * 0.3 + R_z * 0.2
        
        new_R = np.dot(rot, initial_R)
        new_T = np.dot(rot, initial_T)
        
        cam = Camera(
            colmap_id=0,
            R=new_R,
            T=new_T,
            FoVx=start_cam.FoVx,
            FoVy=start_cam.FoVy,
            image=start_cam.original_image,
            gt_alpha_mask=None,
            image_name=start_cam.image_name,
            uid=0)
        
        with torch.no_grad():
            rendering = render(cam, gaussians, pipeline, background)["render"]
        
        img = Image.fromarray((rendering.cpu().numpy() * 255).astype(np.uint8).transpose(1, 2, 0))
        img.save(f"output_frames/{idx:05}.png")
        images.append(img.resize((400, 300)))
    images[0].save('rotating_view.gif', save_all=True, append_images=images[1:], optimize=False, duration=120)
    with open('rotating_view.gif', 'rb') as f:
        display(IPImage(data=f.read(), format='png'))

preview_arc(views[50])

In [ ]:
features = [gaussians._xyz, gaussians._features_dc, gaussians._features_rest, 
            gaussians._scaling, gaussians._rotation, gaussians._opacity]
names = ["xyz", "features_dc", "features_rest", "scaling", "rotation", "opacity"]

for feature, name in zip (features, names):
    display(f"Feature: {name}, Shape: {feature.shape}, Min: {feature.min()}, Max: {feature.max()}")

In [ ]:
first_gaussian_dc = gaussians._features_dc[0, 0, :].cpu().detach().numpy()
first_gaussian_rest = gaussians._features_rest[0, :, :].cpu().detach().numpy()

display(f"DC RGB color: {first_gaussian_dc}")

for i in range(15):
    sh_coeff = first_gaussian_rest[i, :]
    display(f"{i+1}th SH coeff: {sh_coeff}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import sph_harm

def evaluate_color_1(sh_coeffs, theta, phi, deg):
    color = np.zeros(3)
    index = 0
    for l in range(0, 4):
        for m in range(-l, l+1):
            Y = sph_harm(m, l, phi, theta)
            color += sh_coeffs[index] * np.real(Y)
            index += 1
    return np.clip(color, 0, 1)

C0 = 0.28209479177387814
C1 = 0.4886025119029199
C2 = [
    1.0925484305920792,
    -1.0925484305920792,
    0.31539156525252005,
    -1.0925484305920792,
    0.5462742152960396
]
C3 = [
    -0.5900435899266435,
    2.890611442640554,
    -0.4570457994644658,
    0.3731763325901154,
    -0.4570457994644658,
    1.445305721320277,
    -0.5900435899266435
]
C4 = [
    2.5033429417967046,
    -1.7701307697799304,
    0.9461746957575601,
    -0.6690465435572892,
    0.10578554691520431,
    -0.6690465435572892,
    0.47308734787878004,
    -1.7701307697799304,
    0.6258357354491761,
]

def evaluate_color_2(sh, theta, phi, deg):
    color = np.zeros(3)
    color = C0 * sh[0]
    if deg > 0:
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)
        color = (color - 
                 C1 * y * sh[1] +
                 C1 * z * sh[2] -
                 C1 * x * sh[3])
        
        if deg > 1 :
            xx, yy, zz = x * x, y * y, z * z
            xy, yz, xz = x * y, y * z, x * z
            color = (color +
                    C2[0] * xy * sh[4] +
                    C2[1] * yz * sh[5] +
                    C2[2] * (2.0 * zz - xx - yy) * sh[6] +
                    C2[3] * xz * sh[7] +
                    C2[4] * (xx - yy) * sh[8])
            
            if deg > 2:
                color = (color +
                C3[0] * y * (3 * xx - yy) * sh[9] +
                C3[1] * xy * z * sh[10] +
                C3[2] * y * (4 * zz - xx - yy)* sh[11] +
                C3[3] * z * (2 * zz - 3 * xx - 3 * yy) * sh[12] +
                C3[4] * x * (4 * zz - xx - yy) * sh[13] +
                C3[5] * z * (xx - yy) * sh[14] +
                C3[6] * x * (xx - 3 * yy) * sh[15])
    return np.clip(color, 0, 1)

def render_sh(g_idx, eval_func, deg, res=100):
    first_gaussian_dc = gaussians._features_dc[g_idx, 0, :].cpu().detach().numpy()
    first_gaussian_rest = gaussians._features_rest[g_idx, :, :].cpu().detach().numpy()
    
    sh_coeffs = np.concatenate(([first_gaussian_dc], first_gaussian_rest), axis=0)
    phi, theta = np.mgrid[0:2*np.pi:res*1j, 0:np.pi:res*1j]
    
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    
    colors = np.zeros([res, res, 3])
    
    for i in range(res):
        for j in range(res):
            colors[i, j] = eval_func(sh_coeffs, theta[i, j], phi[i, j], deg)
            
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=colors, linewidth=0, antialiased=False)
    return colors


In [ ]:
point_id = 500000
colors_1 = render_sh(point_id, evaluate_color_1, 3)
colors_2 = render_sh(point_id, evaluate_color_2, 3)

In [ ]:
clip_loss_fn = CLIPLossToTargets(text_prompts=['A van gogh painting in the style of starry night'])
with torch.no_grad():
    l = clip_loss_fn(rendering.unsqueeze(0))
l

In [ ]:
from tqdm.auto import tqdm
from IPython.display import display, clear_output

!rm -rf output_frames
!mkdir -p output_frames

opt = torch.optim.Adam(
    [
        # gaussians._xyz,
        gaussians._features_dc,
        gaussians._features_rest,
        # gaussians._scaling,
        # gaussians._rotation,
        # gaussians._opacity
    ], lr=1e-3
)
for i in tqdm(range(30)):
    for view in views:
        opt.zero_grad()
        rendering = render(view, gaussians, pipeline, background)["render"]
        gt = view.original_image[0:3, :, :]
        clip_loss = clip_loss_fn(rendering.unsqueeze(0)) * 5
        loss = clip_loss
        loss.backward()
        opt.step()
    
    p = preview(views[0])
    p.save(f"output_frames/{i:05}.png")
    clear_output(wait=True)
    display(p)

In [ ]:
!ffmpeg -y -r 5 -i output_frames/%05d.png -c:v libx264 -vf fps=25 -pix_fmt yuv420p output.mp4

from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
        <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
preview_arc(views[40])